In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Il Divo,When A Child Is Born,[Sebastien:] \r\nA ray of hope flickers in th...
1,Frankie Goes To Hollywood,Relax,"Guess what's happening now? \r\nHey hey, whoa..."
2,Our Lady Peace,Home,"Way back, and if you find out again \r\nI'll ..."
3,Bruce Springsteen,Empty Sky,I woke up this morning \r\nI could barely bre...
4,Noa,Homemade Religion,Stop. \r\nIt' s 3: 15 \r\nYou have one minut...
5,Offspring,Dirty Magic,In my own simple way \r\nI think she wants me...
6,Rolling Stones,I'm All Right,I wanna tell you something baby \r\nThat you ...
7,Backstreet Boys,Something That I Already Know,"Here we are, seven days \r\nAnd seven nights ..."
8,Peter Cetera,Faithfully,Some say it's just the nature of the beast \r...
9,Noa,By The Light Of The Moon,By the light of the moon \r\nI sing to you \...


In [9]:
df['text'][0]

"[Sebastien:]  \r\nA ray of hope flickers in the sky  \r\nA tiny star lights up way up high  \r\n  \r\n[Urs:]  \r\nAll across the land dawns a brand new morn',  \r\nThis comes to pass when a child is born.  \r\n  \r\n[Carlos:]  \r\nA silent wish sails the seven seas  \r\nThe winds of change whisper in the trees  \r\n  \r\n[David:]  \r\nAnd the walls of doubt crumble tossed and torn,  \r\nThis comes to pass when a child is born.  \r\n  \r\nIt's all a dream, an illusion now.  \r\nIt must come true some time soon somehow  \r\nAll across the land dawns a brand new morn',  \r\nThis comes to pass when a child is born.  \r\nWhen a child is born.\r\n\r\n"

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
import nltk
from nltk.tokenize import word_tokenize

# Download necessary NLTK data if not already downloaded
nltk.download('punkt')

# Define tokenization function
def tokenization(text):
    return word_tokenize(text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\theni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df['text'] = df['text'].apply(lambda x: tokenization(x))


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [18]:
similarity[0]

array([1.        , 0.00912009, 0.00217367, ..., 0.01833674, 0.00504093,
       0.0098795 ])

In [19]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [20]:
def recommendation(song_df):
    # Check if the song exists in the DataFrame
    if song_df in df['song'].values:
        # Get the index of the song
        idx = df[df['song'] == song_df].index[0]
        
        # Calculate distances and sort by similarity
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

        # Recommend the top 5 most similar songs
        songs = []
        for i in distances[1:6]:  # Skipping the first because it's the same song
            songs.append(df.iloc[i[0]]['song'])
        
        return songs
    else:
        # If the song is not found, return a message
        return f"'{song_df}' "


In [21]:
recommendation('Crying Over You')

"'Crying Over You' "

In [22]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))